In [1]:
#imports
import cv2
import numpy as np

In [2]:
# Detect Speed of face in 2D video

def calcSpeed2D(prev_faces, faces, num_points, frame):

    '''
    This function calculates a rough equivalent of the instantaneous 2D speed of the detected faces in a video using the velocity 
    formula. For each face, a number of equidistant points (chosen by the user, minimum of 4), are sampled from the bounding box
    and the velocity is calculated from the number of pixels traveled (on average?) by each point of interest. 

    Inputs:
        - prev_faces: list of bounding box coordinates (x, y, w, h) of faces in the most recent previous sampled state
        - faces: list of bounding box coordinates (x, y, w, h) of faces in the current sampled state
        - num_points (int): number of points on bounding box to be selected (minimum of 4, must be a multiple of 4)

    Outputs:
        - None: simply displays the velocity under the bounding box of the face
    '''

    assert (num_points >= 4) and (num_points % 4 == 0), "Number of points sampled must be a positive, non-zero multiple of 4."


    for (xp, yp, wp, hp), (xc, yc, wc, hc) in zip(prev_faces, faces):

        cv2.rectangle(frame, (xc, yc), (xc+wc, yc+hc), (255, 0, 0), 2)

        # sample points of interest on the bounding boxes

        pPois = []
        cPois = []

        bisectors = num_points // 4
        for i in np.arange(bisectors):
            for j in np.arange(bisectors):
                if (i != 0 or i != bisectors-1) or (j != 0 or j != bisectors - 1): # if point would not be on edge of bounding box
                    continue
                else:
                    pPois.append((xp + i*wp/bisectors, yp + j*hp/bisectors))
                    cPois.append((xc + i*wc/bisectors, yc + j*hc/bisectors))
            
        # calculate the differences between the previous and current points of interest
        diff = np.subtract(pPois, cPois)

        # compute the average difference of each point
        avg_diff = np.mean(diff)

        # calculate the speed based on the difference (using 30 fps as the framerate)
        velo = np.linalg.norm(avg_diff) * 30

        # create the output string to go under the bounding box on the video
        out_str = 'Velocity: ' + str(velo) + ' pixels/sec'

        cv2.putText(frame, out_str, (xc, yc + hc + 20), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 0, 0), 1)

In [4]:
#load in the videos
cap = cv2.VideoCapture('./test-0-color.mp4')

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

fCount = 0

prev_faces = []

if (cap.isOpened()== False):
    print("error opening")
    
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret:

        faces = face_cascade.detectMultiScale(frame, 1.1, 4)
        
        calcSpeed2D(prev_faces, faces, 4, frame)          
        
        cv2.imshow('frame', frame)

        prev_faces = faces
        fCount += 1
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()


In [2]:
# Function that is able to detect face of frame in video

In [ ]:
# Detect speed of face using depth data
# This will be tied to the distance of nose